In [1]:
#import sys
#from dotenv import load_dotenv
import requests
from crypto import (
    convertDataToJSON, 
    pinJSONtoIPFS, 
    initContract, 
    w3,
)

In [2]:
#import os
#from dotenv import load_dotenv
#load_dotenv()
#os.getenv("PINATA_API_KEY")
#os.getenv("PINATA_SECRET_API_KEY")

In [3]:
contract_address = "0x6c09860051F3958c4Bf3D02e2B05e13862D314D7"
abi_file = "FarmsMarket.json"

In [4]:
fm_contract = initContract(contract_address, abi_file)

Contract Address 0x6c09860051F3958c4Bf3D02e2B05e13862D314D7


In [5]:
# updateProduct(uint productID, string memory _type, uint vendorID, string memory URI, uint quantity, uint price) 
#    public returns(uint)
# function removeProduct(uint productID) public returns(uint)
# function makePurchase(uint purchaseDate, uint deliveryDate, uint vendorID, uint productID, uint quantity) 
#    public returns(uint)

In [6]:
#    struct vendor {
#        string URI;
#        address _address;
#    }
def vendor_info():
    name = input("Enter the Name: ")
    phone = input("Enter a Contact Phone: ")
    
    vendor_object = {
            "name": name,
            "phone": phone,
    }

    return vendor_object

# registerVendor(address vendorAddress, string memory vendorURI) 
# public returns(uint)

def register_vendor(vendor_address:str):

    # use contact_info to create URI using pinata API
    vendor_object = vendor_info()
    data = convertDataToJSON(vendor_object)
    ipfs_link = pinJSONtoIPFS(data)

    # call contract to create new owner
    tx_hash = fm_contract.functions.registerVendor(
        vendor_address, ipfs_link)\
        .transact({"from": w3.eth.accounts[0]})

    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt

In [7]:
#    struct product {
#        string _type;
#        uint vendorID;
#        string URI; 
#        uint quantity;
#        uint price;
#    }

def createProduct():
    _type = input("Enter the product type: ")
    quantity = input("Enter the pruduct quantity: ")
    price = input("Enter the pruduct price:")
    
    product_object = {
            "_type": _type,
            "quantity": quantity,
            "price": price,
    }

    return product_object

# registerProduct (string memory _type, uint vendorID, 
#                 string memory URI, uint quantity, uint price) 
#    public returns(uint)

def register_product(product_info: dict, vendor_address:str):

    # use product_info to create URI using pinata API
    data = convertDataToJSON(product_info)
    ipfs_link = pinJSONtoIPFS(data)

    # call contract to create new owner
    tx_hash = fm_contract.functions.registerProduct(
        vendor_address, ipfs_link)\
        .transact({"from": w3.eth.accounts[0]})

    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt

In [12]:
def getLatestVendor():
    ipfs_hash = fm_contract.functions.getLatestVendor().call()
    message = requests.get(f"https://cloudflare-ipfs.com/ipfs/{ipfs_hash}")
    return message.json()

In [13]:
def getVendorByID(vendor_id):
    ipfs_hash = fm_contract.functions.vendors(vendor_id).call()
    message = requests.get(f"https://cloudflare-ipfs.com/ipfs/{ipfs_hash}")
    return message.json()

In [11]:
option = input("Would you like to get or add a vendor?")

if option == "add":
    vendor_address = "0x1178d2924a873C256B430b1751B43258d7BCE6CE"
    register_vendor(vendor_address)

if option == "get":
    vendor_id = input("Enter a vendor ID to retive the vendor. Hit enter to get the latest.")

    if (vendor_id):
        print(getVendorByID(int(vendor_id)))
    else:
        print(getLatestVendor())

Would you like to get or add a vendor? get
Enter a vendor ID to retive the vendor. Hit enter to get the latest. 


NameError: name 'smart_contract' is not defined